# DNBLab Jupyter Notebook Tutorial

## OAI(2) - Schnittstellenabfragen, Datenauslieferung und Ergebnisanzeige

Diese DNBLab-Tutorials beschreiben Beispielabfragen über die OAI2- und OAI-Schnittstelle mit Python. In der Jupyter Notebook Umgebung kann der dokumentierte Code direkt ausgeführt und angepasst werden. Die Tutorials umfassen die exemplarische Anfrage und Ausgabe der Daten in den verschiedenen Metadatenformaten zur weiteren Verarbeitung.

**Inhaltsverzeichnis:** 


* [Einrichten der Arbeitsumgebung](#API1)
* [OAI2 - Schnittstelle ](#API2)
* [OAI - Schnittstelle ](#API3)

## Einrichten der Arbeitsumgebung <a class="anchor" id="API1"></a>

Um die Arbeitsumgebung für die folgenden Schritte passend einzurichten, sollten zunächst die benötigten Python-Bibliotheken importiert werden. Für Anfragen über die OAI(2)-Schnittstellen wird sickle https://sickle.readthedocs.io/en/latest/tutorial.html und zur Verarbeitung der XML-Daten etree https://docs.python.org/3/library/xml.etree.elementtree.html verwendet. Mit dem OAIRecordReader https://polymatheia.readthedocs.io/en/latest/api/polymatheia_data_reader.html können Elemente aus dem METS/MODS Format ausgelesen werden.

In [ ]:
from sickle import Sickle
from lxml import etree

## OAI2- Schnittstelle <a class="anchor" id="API2"></a>

Inhaltsverzeichnis:

* [1. Grundlegende Funktionen](#Teil1)
* [2. Ausgabe von Titeln eines Projekts ab bestimmtem Zeitraum](#Teil2)
* [3. Zusätzliche Ausgabe der URNs/Links zu den Objekten](#Teil3)
* [4. Ausgabe der gescannten Bilder](#Teil4)




### 1. Grundlegende Funktionen <a class="anchor" id="Teil1"></a>

Die Daten der frei verfügbaren digitalisierten Objekte werden über die OAI2-Schnittstelle standardmäßig als XML-Antwort in dem Format METS/MODS ausgeliefert. Zusätzlich steht die Auslieferung der Daten in Dublin Core (DC) zur Verfügung.

Die OAI2 Basis URL ist https://services.dnb.de/oai2/repository. 

In [ ]:
sickle = Sickle('https://services.dnb.de/oai2/repository')

Im nächsten Schritt werden alle verfügbaren Objektsammlungen (Projekte) mit der Funktion ListSets abgefragt und angezeigt.

In [ ]:
oai_sets = sickle.ListSets()
for oai_set in oai_sets:
    print('setSpec value for selective harvesting: ' + oai_set.setSpec)
    print('Name of the set (setName): ' + oai_set.setName + '\n')

Die bisher zur Verfügung stehenden freien Objektsammlungen entsprechen folgenden Projekten (oai_sets):
 
| oai_set | Beschreibung | Anzeige im Katalog |
| --- | --- | --- |
| dnb:digitalisate-oa:projekt4 | 100.000 Seiten aus 30 Titeln, die ein breites Spektrum der Exilpresse 1933-1945 repräsentieren | <a href="https://portal.dnb.de/opac.htm?method=showPreviousResultSite&currentResultId=%22exilpresse%22+and+%22digital%22%26any%26dnb.dea.exp%26leipzig&currentPosition=10">Sammlung im Katalog</a>
| dnb:digitalisate-oa:projekt5 | 1.436 Hefte von 26 in NS-Deutschland erschienenen jüdischen Periodika | <a href="https://portal.dnb.de/opac.htm?method=moveDown&currentResultId=cod%3Dd005+and+location%3Donlinefree%26any&categoryId=dnb.dea.jup">Sammlung im Katalog</a>
| dnb:digitalisate-oa:projekt7 | Gemeinfreie Digitalisate der Exilsammlungen | <a href="https://portal.dnb.de/opac.htm?query=cod%3Dd007+location%3Donlinefree+&method=simpleSearch&cqlMode=true">Sammlung im Katalog</a>
| dnb:digitalisate-oa:projekt8 | Mehr als 1.200 Bücher, Broschüren, Plakate und andere Einblattdrucke zum Weltkrieg 1914-1918 | <a href="https://portal.dnb.de/opac.htm?query=cod%3Dd008+location%3Donlinefree+&method=simpleSearch&cqlMode=true">Sammlung im Katalog</a>
| dnb:digitalisate-oa:projekt26 | Mehr als 600 digitalisierte Quellen zur deutschen Revolution 1918/1919 und zur Nachkriegszeit | <a href="https://portal.dnb.de/opac.htm?query=cod%3Dd026+location%3Donlinefree+&method=simpleSearch&cqlMode=true">Sammlung im Katalog</a>
| dnb:digitalisate-oa:projekt28 | Digitalisierte Werke aus Architektur, Design, Malerei und Druckkunst von 1918 bis 1933 | <a href="https://portal.dnb.de/opac.htm?query=cod%3Dd028+location%3Donlinefree+&method=simpleSearch&cqlMode=true">Sammlung im Katalog</a>
| dnb:digitalisate-oa:projekt29 | Mehr als 2.400 Bildnisse von Buchhändlern, Buchdruckern und Verlegern des 17. bis 20. Jahrhunderts | <a href="https://portal.dnb.de/opac.htm?query=cod%3Dd029+location%3Donlinefree+&method=simpleSearch&cqlMode=true">Sammlung im Katalog</a>

Die verfügbaren Metadatenformate können mit der Funktion ListMetadataFormats abgefragt und mit print() ausgegeben werden. 


In [ ]:
oai_formats = sickle.ListMetadataFormats()
for oai_format in oai_formats:
    print(oai_format.metadataPrefix)

### 2. Ausgabe von Titeln eines Projekts in bestimmten Zeitraum <a class="anchor" id="Teil2"></a>

Um beispielhaft alle Datensätze von projekt28 ab dem 01.04.2019 aufzulisten, müssen für die Abfrage Metadatenformat, Zeitraum und Projekt angegeben werden.
Die OAI Funktionen werden unter http://www.openarchives.org/OAI/2.0/ spezifiziert und auf der DNBLab Seite unter Funktionen beschrieben.
Die Daten befinden sich in der XML-Antwort als Inhalt zwischen beschreibenden Elementen und werden durchnummeriert angezeigt. Für die gezielte Abfrage der gewünschten Informationen aus dem Metadatenformat DC können bei Bedarf die DC-Spezifikationen http://www.openarchives.org/OAI/2.0/oai_dc/ und http://purl.org/dc/elements/1.1/ herangezogen werden.


In [ ]:
namespaces = {
    'oai': 'http://www.openarchives.org/OAI/2.0/',
    'oai_dc': 'http://www.openarchives.org/OAI/2.0/oai_dc/',
    'dc': 'http://purl.org/dc/elements/1.1/'
}

count = 0
#max_results = 10

for record in sickle.ListRecords(**{'metadataPrefix': 'oai_dc','from': '2019-04-30', 'set': 'dnb:digitalisate-oa:projekt28'}):
    if 'ger' in record.raw:
        tree = etree.ElementTree(record.xml)
        result = tree.xpath('/oai:record/oai:metadata/oai_dc:dc/dc:title/text()', namespaces=namespaces)
        if result:
            count += 1
            author = tree.xpath('/oai:record/oai:metadata/oai_dc:dc/dc:creator/text()', namespaces=namespaces)
            print(f"{count}: {result[0]}")
            if author:
                print(author[0])
            
         #   if count >= max_results:
           #     break


### 3. Zusätzliche Ausgabe der Links zu den Objekten <a class="anchor" id="Teil3"></a>

Damit zusätzlich zu den Titeln die Links zu den Objekten ausgegeben werden, wird der URN entsprechend des dc-Schemas /oai:record/oai:metadata/oai_dc:dc/dc:identifier/text() angesprochen und zusammen mit dem Titel angezeigt. 

In [ ]:
namespaces = {
    'oai': 'http://www.openarchives.org/OAI/2.0/',
    'oai_dc': 'http://www.openarchives.org/OAI/2.0/oai_dc/',
    'dc': 'http://purl.org/dc/elements/1.1/'
}

count = 0
max_results = 10

for record in sickle.ListRecords(**{'metadataPrefix': 'oai_dc', 'from': '2019-04-30', 'set': 'dnb:digitalisate-oa:projekt28'}):
    if count >= max_results:
        break
    
    if ('ger' in record.raw):
        tree = etree.ElementTree(record.xml)
        result = tree.xpath('/oai:record/oai:metadata/oai_dc:dc/dc:title/text()', namespaces=namespaces)
        if (result):
            count += 1
            urn = tree.xpath('/oai:record/oai:metadata/oai_dc:dc/dc:identifier/text()', namespaces=namespaces)
            print(f"{count}: {result[0]}")
            if urn:
                print(f"https://nbn-resolving.org/{urn[0]}")

    if count >= max_results:
        break


### 4. Ausgabe der gescannten Bilder <a class="anchor" id="Teil4"></a>

Das Format METS bildet neben den bibliographischen Daten im Format MODS die Struktur der digitalen Objekte bzw. die Reihenfolge der gescannten Bilder ab. Für die gezielte Abfrage der gewünschten Informationen aus dem Metadatenformat METS können bei Bedarf die METS-Spezifikationen http://www.loc.gov/METS/ und https://www.loc.gov/standards/mets/mets.xsd herangezogen werden. Die Links der einzelnen Bilder (Thumbnails und Defaultbilder) können über <xsd:element name="fileGrp"> angesprochen und die ersten 10 zur weiteren Verarbeitung ausgegeben werden.

In [ ]:
import requests
import xml.etree.ElementTree as ET

# Funktion zum Abrufen der OAI2-Daten
def fetch_oai_records(url, max_records=10, metadata_prefix='mets', set_spec='dnb:digitalisate-oa:projekt28'):
    params = {
        'verb': 'ListRecords',
        'metadataPrefix': metadata_prefix,
        'set': set_spec,
        'resumptionToken': None
    }
    records = []
    
    while len(records) < max_records:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Überprüfen, ob die Anfrage erfolgreich war
        xml_data = response.content
        records.append(xml_data)

        # Überprüfen, ob es einen Resumption Token gibt
        root = ET.fromstring(xml_data)
        resumption_token = root.find('.//{http://www.openarchives.org/OAI/2.0/}resumptionToken')
        
        if resumption_token is not None:
            params['resumptionToken'] = resumption_token.text
        else:
            break

    return records[:max_records]

# Hauptfunktion
def main():
    url = 'http://services.dnb.de/oai2/repository'
    xml_records = fetch_oai_records(url)

    # Namespace für METS definieren
    ns_mets = {'mets': 'http://www.loc.gov/METS/'}
    ns_oai = {'oai': 'http://www.openarchives.org/OAI/2.0/'}

    count = 0
    # Durch die Datensätze iterieren
    for xml_data in xml_records:
        root = ET.fromstring(xml_data)
        
        for record in root.findall('.//oai:record', ns_oai):
            if count >= 10:
                return
            mets_file_sec = record.find('.//mets:fileSec', ns_mets)
            if mets_file_sec is not None:
                mets_file_grp = mets_file_sec.find('.//mets:fileGrp', ns_mets)
                if mets_file_grp is not None:
                    if isinstance(mets_file_grp, list):
                        # Wenn es eine Liste ist, den ersten Eintrag ausgeben
                        mets_file = mets_file_grp[0].find('.//mets:file', ns_mets)
                        if mets_file is not None:
                            flocat = mets_file.find('.//mets:FLocat', ns_mets)
                            if flocat is not None:
                                href = flocat.attrib.get('{http://www.w3.org/1999/xlink}href')
                                if href:
                                    print(href)
                                    count += 1
                    else:
                        # Ansonsten direkt ausgeben
                        mets_file = mets_file_grp.find('.//mets:file', ns_mets)
                        if mets_file is not None:
                            flocat = mets_file.find('.//mets:FLocat', ns_mets)
                            if flocat is not None:
                                href = flocat.attrib.get('{http://www.w3.org/1999/xlink}href')
                                if href:
                                    print(href)
                                    count += 1
            if count >= 10:
                return

if __name__ == "__main__":
    main()


Inhaltsverzeichnis:

* [1. Grundlegende Funktionen](#Teil5)
* [2. Ausgabe von Titeln in bestimmten Zeitraum](#Teil6)
* [3. Zusätzliche Ausgabe der Links zu den Objekten](#Teil7)

### 1. Grundlegende Funktionen <a class="anchor" id="Teil5"></a>

Die Daten werden über die OAI-Schnittstelle standardmäßig als XML-Antwort in verschiedenen Formaten ausgeliefert.

Die OAI Basis URL ist https://services.dnb.de/oai/repository.

In [ ]:
sickle = Sickle('https://services.dnb.de/oai/repository')

Im nächsten Schritt werden alle verfügbaren Datensets (Kataloge) mit der Funktion ListSets abgefragt und angezeigt.

In [ ]:
oai_sets = sickle.ListSets()
for oai_set in oai_sets:
    print('setSpec value for selective harvesting: ' + oai_set.setSpec)
    print('Name of the set (setName): ' + oai_set.setName + '\n')

setSpec value for selective harvesting: dnb:wv:reiheT:sg220
Name of the set (setName): Deutsche Nationalbibliografie (nur Datensaetze nach abgeschlossener Bearbeitung): Musiktontraeger - Bibel

setSpec value for selective harvesting: dnb:wv:reiheT:sg230
Name of the set (setName): Deutsche Nationalbibliografie (nur Datensaetze nach abgeschlossener Bearbeitung): Musiktontraeger - Theologie, Christentum

setSpec value for selective harvesting: dnb:wv:reiheT:sg290
Name of the set (setName): Deutsche Nationalbibliografie (nur Datensaetze nach abgeschlossener Bearbeitung): Musiktontraeger - Andere Religionen

setSpec value for selective harvesting: dnb:wv:reiheT:sg3
Name of the set (setName): Deutsche Nationalbibliografie (nur Datensaetze nach abgeschlossener Bearbeitung): Musiktontraeger - Sozialwissenschaften

setSpec value for selective harvesting: dnb:wv:reiheT:sg300
Name of the set (setName): Deutsche Nationalbibliografie (nur Datensaetze nach abgeschlossener Bearbeitung): Musiktontraeg

setSpec value for selective harvesting: dnb-appr:reiheC:sgB
Name of the set (setName): Deutsche Nationalbibliografie: Karten - Belletristik

setSpec value for selective harvesting: dnb-appr:reiheC:sgK
Name of the set (setName): Deutsche Nationalbibliografie: Karten - Kinder- und Jugendliteratur

setSpec value for selective harvesting: dnb-appr:reiheC:sgS
Name of the set (setName): Deutsche Nationalbibliografie: Karten - Schulbücher

setSpec value for selective harvesting: dnb-appr:reiheC:sgY
Name of the set (setName): Deutsche Nationalbibliografie: Karten - Abgeschlossene Veröffentlichungen

setSpec value for selective harvesting: dnb-appr:reiheC:sgZ
Name of the set (setName): Deutsche Nationalbibliografie: Karten - Berichtigungen

setSpec value for selective harvesting: dnb-appr:reiheC:holdings
Name of the set (setName): Deutsche Nationalbibliografie: Karten - Bestandssätze

setSpec value for selective harvesting: dnb-appr:reiheH
Name of the set (setName): Deutsche Nationalbibliografi

setSpec value for selective harvesting: dnb-appr:wv:reiheA
Name of the set (setName): Deutsche Nationalbibliografie (nur Datensaetze nach abgeschlossener Bearbeitung): Publikationen des Verlagsbuchhandels

setSpec value for selective harvesting: dnb-appr:wv:reiheA:sg0
Name of the set (setName): Deutsche Nationalbibliografie (nur Datensaetze nach abgeschlossener Bearbeitung): Publikationen des Verlagsbuchhandels - Allgemeines, Informationswissenschaft

setSpec value for selective harvesting: dnb-appr:wv:reiheA:sg000
Name of the set (setName): Deutsche Nationalbibliografie (nur Datensaetze nach abgeschlossener Bearbeitung): Publikationen des Verlagsbuchhandels - Allgemeines, Wissenschaft

setSpec value for selective harvesting: dnb-appr:wv:reiheA:sg004
Name of the set (setName): Deutsche Nationalbibliografie (nur Datensaetze nach abgeschlossener Bearbeitung): Publikationen des Verlagsbuchhandels - Informatik

setSpec value for selective harvesting: dnb-appr:wv:reiheA:sg010
Name of the se


setSpec value for selective harvesting: dnb-appr:wv:reiheM:sg230
Name of the set (setName): Deutsche Nationalbibliografie (nur Datensaetze nach abgeschlossener Bearbeitung): Musikalien - Theologie, Christentum

setSpec value for selective harvesting: dnb-appr:wv:reiheM:sg290
Name of the set (setName): Deutsche Nationalbibliografie (nur Datensaetze nach abgeschlossener Bearbeitung): Musikalien - Andere Religionen

setSpec value for selective harvesting: dnb-appr:wv:reiheM:sg3
Name of the set (setName): Deutsche Nationalbibliografie (nur Datensaetze nach abgeschlossener Bearbeitung): Musikalien - Sozialwissenschaften

setSpec value for selective harvesting: dnb-appr:wv:reiheM:sg300
Name of the set (setName): Deutsche Nationalbibliografie (nur Datensaetze nach abgeschlossener Bearbeitung): Musikalien - Sozialwissenschaften, Soziologie

setSpec value for selective harvesting: dnb-appr:wv:reiheM:sg310
Name of the set (setName): Deutsche Nationalbibliografie (nur Datensaetze nach abgeschlos

Name of the set (setName): Meldungen der Verleger an das Verzeichnis Lieferbarer Bücher (nicht Bestandteil der Deutschen Nationalbibliografie) - Sprache, Linguistik

setSpec value for selective harvesting: dnb-all-appr:reiheN:sg420
Name of the set (setName): Meldungen der Verleger an das Verzeichnis Lieferbarer Bücher (nicht Bestandteil der Deutschen Nationalbibliografie) - Englisch

setSpec value for selective harvesting: dnb-all-appr:reiheN:sg430
Name of the set (setName): Meldungen der Verleger an das Verzeichnis Lieferbarer Bücher (nicht Bestandteil der Deutschen Nationalbibliografie) - Deutsch

setSpec value for selective harvesting: dnb-all-appr:reiheN:sg439
Name of the set (setName): Meldungen der Verleger an das Verzeichnis Lieferbarer Bücher (nicht Bestandteil der Deutschen Nationalbibliografie) - Andere germanische Sprachen

setSpec value for selective harvesting: dnb-all-appr:reiheN:sg440
Name of the set (setName): Meldungen der Verleger an das Verzeichnis Lieferbarer Bücher

setSpec value for selective harvesting: zdb:online:sg940
Name of the set (setName): Online Publikationen der ZDB - Geschichte Europas

setSpec value for selective harvesting: zdb:online:sg943
Name of the set (setName): Online Publikationen der ZDB - Geschichte Deutschlands

setSpec value for selective harvesting: zdb:online:sg950
Name of the set (setName): Online Publikationen der ZDB - Geschichte Asiens

setSpec value for selective harvesting: zdb:online:sg960
Name of the set (setName): Online Publikationen der ZDB - Geschichte Afrikas

setSpec value for selective harvesting: zdb:online:sg970
Name of the set (setName): Online Publikationen der ZDB - Geschichte Nordamerikas

setSpec value for selective harvesting: zdb:online:sg980
Name of the set (setName): Online Publikationen der ZDB - Geschichte Südamerikas

setSpec value for selective harvesting: zdb:online:sg990
Name of the set (setName): Online Publikationen der ZDB - Geschichte der übrigen Welt

setSpec value for selective harve

Name of the set (setName): Online Publikationen ohne Einschraenkungen - Schulbücher

setSpec value for selective harvesting: dnb-all:online:sgY
Name of the set (setName): Online Publikationen ohne Einschraenkungen - Abgeschlossene Veröffentlichungen

setSpec value for selective harvesting: dnb-all:online:sgZ
Name of the set (setName): Online Publikationen ohne Einschraenkungen - Berichtigungen

setSpec value for selective harvesting: dnb-all:online:holdings
Name of the set (setName): Online Publikationen ohne Einschraenkungen - Bestandssätze

setSpec value for selective harvesting: dnb-all:online:dissertations
Name of the set (setName): Online Dissertationen

setSpec value for selective harvesting: dnb-all:online:dissertations:sg0
Name of the set (setName): Online Dissertationen - Allgemeines, Informationswissenschaft

setSpec value for selective harvesting: dnb-all:online:dissertations:sg000
Name of the set (setName): Online Dissertationen - Allgemeines, Wissenschaft

setSpec value fo

Hier eine Auswahl der über OAI verfügbaren Titeldaten (oai_sets):

| Auswahl | Wert für Parameter "set" | 
| --- | --- | 
| Deutsche Nationalbibliografie ohne Gemeinsame Normdatei| dnb:wv (nur Datensätze nach abgeschlossener Bearbeitung) | 
| Deutsche Nationalbibliografie, Reihe A (Publikationen des Verlagsbuchhandels) | dnb:reiheA (inkl. Datensätze in Bearbeitung und abgeschlossene Bearbeitung) |
| Deutsche Nationalbibliografie, Reihe B (Publikationen außerhalb des Verlagsbuchhandels) | dnb:reiheB (inkl. Datensätze in Bearbeitung und abgeschlossene Bearbeitung)
dnb:wv:reiheB (nur Datensätze nach abgeschlossener Bearbeitung) |
| Deutsche Nationalbibliografie, Reihe H (Hochschulschriften)| dnb:reiheH (inkl. Datensätze in Bearbeitung und abgeschlossene Bearbeitung)
dnb:wv:reiheH (nur Datensätze nach abgeschlossener Bearbeitung) |
| Deutsche Nationalbibliografie, Reihe O (Online-Publikationen) | dnb:reiheO (inkl. Datensätze in Bearbeitung und abgeschlossene Bearbeitung)
dnb:wv:reiheO (nur Datensätze nach abgeschlossener Bearbeitung) |
| Online-Publikationen ohne Einschränkungen | dnb-all:online |
| Deutsche Nationalbibliografie: Digitalisierte Inhaltsverzeichnisse| dnb:toc |

Die Daten können über die OAI-Schnittstelle als XML-Antwort in folgenden Formaten ausgeliefert werden: 
* MARC21-xml http://www.loc.gov/standards/marcxml/schema/MARC21slim.xsd
* DNB Casual (oai_dc)	http://www.openarchives.org/OAI/2.0/oai_dc.xsd
* RDF (RDFxml) http://www.w3.org/2000/07/rdf.xsd

### 2. Ausgabe von Titeln eines Projekts in bestimmten Zeitraum <a class="anchor" id="Teil6"></a>
Um beispielhaft Online-Dissertationen aus der Sachgruppe „Sozialwissenschaften, Soziologie, Anthropologie“ ab dem 01.04.2019 aufzulisten, müssen für die Abfrage Metadatenformat, Zeitraum und Projekt angegeben werden. Die OAI Funktionen werden unter http://www.openarchives.org/OAI/2.0/ spezifiziert und auf der DNBLab Seite unter Funktionen beschrieben. Die Daten befinden sich in der XML-Antwort als Inhalt zwischen beschreibenden Elementen und werden durchnummeriert angezeigt. Für die gezielte Abfrage der gewünschten Informationen aus dem Metadatenformat DC, können bei Bedarf die DC-Spezifikationen http://www.openarchives.org/OAI/2.0/oai_dc/ und http://purl.org/dc/elements/1.1/ herangezogen werden.

In [ ]:
namespaces = {
    'oai': 'http://www.openarchives.org/OAI/2.0/',
    'oai_dc': 'http://www.openarchives.org/OAI/2.0/oai_dc/',
    'dc': 'http://purl.org/dc/elements/1.1/'
}

count=0

for record in sickle.ListRecords(**{'metadataPrefix': 'oai_dc', 'from': '2019-04-30', 'set': 'dnb-all:online:dissertations:sg300'}):
    
    if ('ger' in record.raw):
        tree = etree.ElementTree(record.xml)
        result = tree.xpath('/oai:record/oai:metadata/oai_dc:dc/dc:title/text()', namespaces=namespaces)
        if (result):
            count += 1
            author = tree.xpath('/oai:record/oai:metadata/oai_dc:dc/dc:creator/text()', namespaces=namespaces)
            print(str(count) + ": " + result[0])
            if author:
                print(author[0])

168: Die politische Repräsentation von Personen verschiedenen sozialen Geschlechts in der Europäischen Union und die europäische Gleichstellungsgesetzgebung : eine Untersuchung der EU-Gesetzgebung zur Gleichstellung der Geschlechter seit dem Inkrafttreten des Vertrages von Maastricht 1993 / Katalin Hahn ; Gutachter: Heike M. Grimm, Andreas Blätte
Hahn, Katalin Verfasser]
169: Gesellschaft verändern! : Konturen einer performativen Soziologie / Robert Jende ; Gutachter: Stephan Lessenich, Hartmut Rosa
Jende, Robert [Verfasser]
170: Regie in der Region : Kooperationen und Konstruktionen sozialer Netzwerkstrukturen in Tourismusregionen / Henryk Balkow ; Gutachter: Frank Ettrich, Peter Walgenbach
Balkow, Henryk [Verfasser]
171: Perspectives on Self-Reported Resilience - Cross-Sectional, Longitudinal and Meta-Analytical Considerations / Sarah K. Schäfer
Schäfer, Sarah K. [Verfasser]
172: Riskantes Gesundheitsverhalten in der allgemeinen Erwachsenenbevölkerung in Deutschland / Josefine Atzend

218: Das Krankenhaus "zwischen" Funktionssystemen und Organisation. Eine systemtheoretische Analyse über die Wirkung bzw. Nicht-Wirkung gesellschaftlicher Erwartungen auf die Strukturen von Krankenhäusern. Systematisches Literaturreview (Teil II) / Barbara Bowert ; Gutachter: Marcus Siebolds, Manfred Borutta ; Betreuer: Marcus Siebolds
Bowert, Barbara [Verfasser]
219: Der Diskurs des Jahres 2012 über die Beschneidung – medizinische vs. religiöse Rationalität : Eine systematische quantitative und qualitative Inhaltsanalyse des medialen Diskurses über die Beschneidung in Print- und Onlinemedien als kritische Diskursanalyse / Maximilian Thormann
Thormann, Maximilian [Verfasser]
220: Den Thatcherismus lehren : Die Vermittlung britischer Kultur im gymnasialen Unterricht / Stefan Alexander Eick
Eick, Stefan Alexander [Verfasser]
221: «Landfremde» an der Adria : Die Deutschen in Triest 1880–1920 / Reinhard Reimann
Reimann, Reinhard [Verfasser]
222: Zukunftsfähigkeit gestalten : Untersuchung n

261: Wandel in der Stadt - Alltagsvorstellungen von Schülerinnen und Schülern zu städtischem Wandel anhand ihres Stadtquartiers / Kristina Rubarth ; Gutachter: Alexandra Budke, Günther Weiss
Rubarth, Kristina [Verfasser]
262: Auswirkungen der Sozialisation im Kontext von Lebens- und Arbeitsalltag ägyptischer Journalistinnen / Schirin Khalik-Hilper ; Gutachter: Kai Hafez, Renate Kreile
Khalik-Hilper, Schirin Verfasser]
263: Bildungsmanagement im Innovationsprozess : Eine fallbezogene Untersuchung der Interaktion von Innovations- und Bildungsaktivitäten in Organisationen / Julia Breßler ; Gutachter: Stefan Hüsig, Peter Pawlowsky ; Betreuer: Stefan Hüsig
Breßler, Julia [Verfasser]
264: (e)Portfolio : eine pädagogische Technologie des Selbst / Stephan Münte-Goussar
Münte-Goussar, Stephan [Verfasser]
265: Technologieakzeptanz Digitaler Medien bei Universitätsstudierenden verschiedener Fächer und Berufstätigen gleichen Alters : eine Studie zu den psychologischen Determinanten und Hintergründ

320: Darf der Staat ohne Ermessen einem Menschen Leid zufügen / Stefanie Wätzel
Wätzel, Stefanie [Verfasser]
321: Design für das Altern im demografischen Wandel in Deutschland - Potenziale zur Einbindung älterer Menschen in zukünftige Lebenswelten / Liesa Anneke König
König, Liesa Anneke [Verfasser]
322: Das Spiel mit der Grenze - Führung im Spannungsfeld von Formalität und Informalität / Timo Hillenbrand ; Gutachter: Andreas Göbel, Veronika Tacke
Hillenbrand, Timo [Verfasser]
323: Lösen von Alltagsproblemen, Kreativität und Sozialkompetenz : interkulturell und intergenerational vergleichende Studie zwischen Deutschland und Südkorea / Seon Joo Born ; Betreuer: Willie van Peer
Born, Seon Joo [Verfasser]
324: Kunstschutz im besetzten Deutschland : Restitution und Kulturpolitik in der französischen und amerikanischen Besatzungszone (1944–1953) / Emily Löffler
Löffler, Emily [Verfasser]
325: Urbaner Wandel und Öffentlichkeit : Die Peripherien Madrids und Barcelonas in der Zeit der Franco-D

382: Verlebendigung der Technik - Perspektiven im feministischen Technikdiskurs / Angelika Saupe ; Gutachter: Marlies Krüger, Maria Osietzki ; Betreuer: Marlies Krüger
Saupe, Angelika [Verfasser]
383: Wissenschaft und Gefühl : Eine Interviewstudie zum Umgang mit Empfindungen in der Forschung an nichtmenschlichen Primaten / Julia Keil
Keil, Julia [Verfasser]
384: Gottschee revisited : Geschichtsnarrative und Identitätsmanagement im Cyberspace / Georg Marschnig
Marschnig, Georg [Verfasser]
385: Die Abwertung des Anderen mittels Sprache im Amateurfußball : Ein soziolinguistischer Vergleich in Deutschland und Frankreich / Florian Koch
Koch, Florian [Verfasser]
386: Russian cities in motion: the Marshrutka as a multifaceted issue of Post-Soviet urban life / Tonio Weicker ; Gutachter: Hans-Liudger Dienel, Oksana Zaporozhets ; Hans-Liudger Dienel, Oksana Zaporozhets
Weicker, Tonio [Verfasser]
387: Zuhause im Quartier - Die räumliche Gestaltung der Umwelt zwecks Förderung der Ortsverbundenheit

...

1318: Spiel-Räume – demokratisch basierte Landesplanung in Mitteldeutschland zwischen 1923 und 1932 sowie 1990 und 2000: Ein Beitrag zur Geschichte und Theorie strategischer, nicht-linearer Planung / Harald Kegler ; Gutachter: Harald Bodenschatz ; Betreuer: Max Welch-Guerra
Kegler, Harald [Verfasser]
1319: Emergenz, Subjekt und Situation. Eine Studie zum Aspekt der Diskontinuität in einer Theorie der autonomen Reproduktion von Gesellschaft / Michael Nguyen. Betreuer: Martina Löw ; Gerhard Gamm
Nguyen, Michael [Verfasser]
1320: Echtzeitpartizipation - Potenziale von sozialen Anwendungen der Kommunikationstechnologien für die Motivation Jugendlicher und junger Erwachsener zur Beteiligung an Partizipationsprozessen in der Stadtentwicklung in Deutschland / Björn Hekmati. Betreuer: Annette Rudolph-Cleff ; Oliver Langbein
Hekmati, Björn [Verfasser]
1321: Schulkultur, Schul- und Qualitätsentwicklung mit neuen Medien. Problemstellungen, Potenziale, Paradoxien. Eine Einzelfallstudie / Claudia Z

1361: Ambivalenzen der Digitalisierung: neue Kommunikations- und Medientechnologien in der Wissenschaft / André Donk. Betreuer: Frank Marcinkowski
Donk, André [Verfasser]
1362: Three essays on the economics of religion / Matthias Opfinger
Opfinger, Matthias [Verfasser]
1363: Nachhaltigkeit und demografischer Wandel : Eine diskursanalytische Betrachtung und ein dynamisches Simulationsmodell als Beitrag zur post-normal Science / Nico Stelljes
Stelljes, Nico [Verfasser]
1364: Die politischen und wirtschaftlichen Beziehungen zwischen der Europäischen Union und Indien : strategische Partnerschaft oder freundliches Desinteresse? / Pierre-Michael Gröning. Betreuer: Wichard Woyke
Gröning, Pierre-Michael [Verfasser]
1365: Förderliche und hemmende Effekte kollektiver Perspektivenübernahme auf den intergruppalen Versöhnungsprozess / Markus Barth
Barth, Markus [Verfasser]
1366: Ressourcen und Belastungen in der Berufsbiografie : Eine Studie zum Zusammenhang zwischen berufsbiografischen Arbeitsbedi

1403: Evaluationsforschung am Beispiel des problemorientierten Lernens : eine Wirkungsanalyse des Modellstudiengangs Medizin am Universitätsklinikum Hamburg-Eppendorf / Maren Kandulla
Kandulla, Maren [Verfasser]
1404: Der zweite Blick in einer reflexiven Geographie und Didaktik : das Lehrstück "Räumliche Mobilität, Ostdeutschland & die ostdeutsche Provinz / Antje Schneider. Gutachter: Tilmann Rhode-Jüchtern ; Benno Werlen ; Christian Vielhaber
Schneider, Antje [Verfasser]
1405: Illness, Media, and Culture : ein interkultureller Vergleich der Darstellung von Allergien in englischen und US-amerikanischen Lifestyle-Magazinen / Andrea Schöndube. Gutachter: Günter Lenz ; Reinhard Isensee
Schöndube, Andrea [Verfasser]
1406: Der Einfluss des Aktivierenden Staates auf Familienpolitik in Deutschland und den Niederlanden / Ansgar Willenborg. Gutachter: Hans Bertram ; Saskia Keuzenkamp
Willenborg, Ansgar [Verfasser]
1407: Kognitive und affektive Messewirkungsforschung : eine empirische Analyse de

1447: IT-Spezialisten in Frankfurt am Main: eine Studie zur Ortsbindung von flexiblen Erwerbstätigen / Thorsten Eger
Eger, Thorsten [Verfasser]
1448: Technisches Erfahrungswissen in industriellen Produktionsprozessen / C Benjamin Nakhosteen
Nakhosteen, C. Benjamin [Verfasser]
1449: Soziale Strukturen und Informationstechnologie : Die organisatorische Bedeutung von „Service Oriented Architectures“ / Sebastian Remer
Remer, Sebastian [Verfasser]
1450: Virtualität und mobile Medientechnologien / Clara Völker ; Betreuer: Lorenz Engell
Völker, Clara [Verfasser]
1451: Charismania - Charisma als "Doping" für Persönlichkeit und Karriere? : eine Untersuchung von sieben Charisma-Ratgebern / Eva Brigitta Müller. Betreuer: Winfried Gebhardt ; Clemens Albrecht
Müller, Eva Brigitta [Verfasser]
1452: De-Gendering informatischer Artefakte: Grundlagen einer kritisch-feministischen Technikgestaltung / Corinna Bath. Gutachter: Susanne Maaß ; Heidi Schelhowe. Betreuer: Susanne Maaß
Bath, Corinna [Verfasser

1531: Wissensorientierte Gestaltung von Wertschöpfungsnetzwerken / Karl Ritsch
Ritsch, Karl [Verfasser]
1532: Ein Beitrag zur Bereitstellung von Wissen und Informationen für eine Produktentwicklung in interkulturellen Unternehmenskooperationen / Matthias Sander
Sander, Matthias [Verfasser]
1533: The Need for Basic Pastoral Formation for Youths in Africa : In Relation to Nigerian Youths of the 21. Century / JoeBarth Ch Abba
Abba, JoeBarth Ch [Verfasser]
1534: Gestaltungsprinzipien für die Initiierung und Realisierung von Public Private Partnership (PPP) an Beispielen der Gewerbeansiedlung und weiterer Dienstleister / Martina Schmette
Schmette, Martina [Verfasser]
1535: Konzentrationsprozess in der Automobilzulieferindustrie : Konsequenzen für das Verhältnis zwischen Hersteller und Zulieferer / Susanne Gmeiner
Gmeiner, Susanne [Verfasser]
1536: Zur Entwicklungsgeschichte und Ästhetik des digitalen Bildes : Von traditionellen Immersionsmedien zum Computerspiel / Andreas Korn
Korn, Andreas

### 3. Zusätzliche Ausgabe der Links zu den Objekten <a class="anchor" id="Teil7"></a>
Damit zusätzlich zu den Titeln die Links zu den Objekten ausgegeben werden, wird der URN entsprechend des dc-Schemas /oai:record/oai:metadata/oai_dc:dc/dc:identifier/text() angesprochen und zusammen mit dem Titel angezeigt.

In [ ]:
namespaces = {
    'oai': 'http://www.openarchives.org/OAI/2.0/',
    'oai_dc': 'http://www.openarchives.org/OAI/2.0/oai_dc/',
    'dc': 'http://purl.org/dc/elements/1.1/'
}

count=0

for record in sickle.ListRecords(**{'metadataPrefix': 'oai_dc', 'from': '2019-04-30', 'set': 'dnb-all:online:dissertations:sg300'}):
    
    if ('ger' in record.raw):
        tree = etree.ElementTree(record.xml)
        result = tree.xpath('/oai:record/oai:metadata/oai_dc:dc/dc:title/text()', namespaces=namespaces)
        if (result):
            count += 1
            urn = tree.xpath('/oai:record/oai:metadata/oai_dc:dc/dc:identifier/text()', namespaces=namespaces)
            print(str(count) + ": " + result[0])
            if urn:
                print("https://nbn-resolving.org/" + urn[0])

168: Die politische Repräsentation von Personen verschiedenen sozialen Geschlechts in der Europäischen Union und die europäische Gleichstellungsgesetzgebung : eine Untersuchung der EU-Gesetzgebung zur Gleichstellung der Geschlechter seit dem Inkrafttreten des Vertrages von Maastricht 1993 / Katalin Hahn ; Gutachter: Heike M. Grimm, Andreas Blätte
https://nbn-resolving.org/urn:nbn:de:gbv:547-202000285
169: Gesellschaft verändern! : Konturen einer performativen Soziologie / Robert Jende ; Gutachter: Stephan Lessenich, Hartmut Rosa
https://nbn-resolving.org/urn:nbn:de:gbv:27-dbt-20200814-085010-001
170: Regie in der Region : Kooperationen und Konstruktionen sozialer Netzwerkstrukturen in Tourismusregionen / Henryk Balkow ; Gutachter: Frank Ettrich, Peter Walgenbach
https://nbn-resolving.org/urn:nbn:de:gbv:547-202000015
171: Perspectives on Self-Reported Resilience - Cross-Sectional, Longitudinal and Meta-Analytical Considerations / Sarah K. Schäfer
https://nbn-resolving.org/urn:nbn:de:bsz

218: Das Krankenhaus "zwischen" Funktionssystemen und Organisation. Eine systemtheoretische Analyse über die Wirkung bzw. Nicht-Wirkung gesellschaftlicher Erwartungen auf die Strukturen von Krankenhäusern. Systematisches Literaturreview (Teil II) / Barbara Bowert ; Gutachter: Marcus Siebolds, Manfred Borutta ; Betreuer: Marcus Siebolds
https://nbn-resolving.org/urn:nbn:de:0295-opus4-17215
219: Der Diskurs des Jahres 2012 über die Beschneidung – medizinische vs. religiöse Rationalität : Eine systematische quantitative und qualitative Inhaltsanalyse des medialen Diskurses über die Beschneidung in Print- und Onlinemedien als kritische Diskursanalyse / Maximilian Thormann
https://nbn-resolving.org/urn:nbn:de:101:1-2020050309313915163530
220: Den Thatcherismus lehren : Die Vermittlung britischer Kultur im gymnasialen Unterricht / Stefan Alexander Eick
https://nbn-resolving.org/urn:nbn:de:101:1-2020050309103634311717
221: «Landfremde» an der Adria : Die Deutschen in Triest 1880–1920 / Reinha

261: Wandel in der Stadt - Alltagsvorstellungen von Schülerinnen und Schülern zu städtischem Wandel anhand ihres Stadtquartiers / Kristina Rubarth ; Gutachter: Alexandra Budke, Günther Weiss
https://nbn-resolving.org/urn:nbn:de:hbz:38-114229
262: Auswirkungen der Sozialisation im Kontext von Lebens- und Arbeitsalltag ägyptischer Journalistinnen / Schirin Khalik-Hilper ; Gutachter: Kai Hafez, Renate Kreile
https://nbn-resolving.org/urn:nbn:de:gbv:547-201900147
263: Bildungsmanagement im Innovationsprozess : Eine fallbezogene Untersuchung der Interaktion von Innovations- und Bildungsaktivitäten in Organisationen / Julia Breßler ; Gutachter: Stefan Hüsig, Peter Pawlowsky ; Betreuer: Stefan Hüsig
https://nbn-resolving.org/urn:nbn:de:bsz:ch1-qucosa2-334201
264: (e)Portfolio : eine pädagogische Technologie des Selbst / Stephan Münte-Goussar
https://nbn-resolving.org/urn:nbn:de:101:1-2020071410132675072090
265: Technologieakzeptanz Digitaler Medien bei Universitätsstudierenden verschiedener F

320: Darf der Staat ohne Ermessen einem Menschen Leid zufügen / Stefanie Wätzel
https://nbn-resolving.org/urn:nbn:de:101:1-2019112923220820108245
321: Design für das Altern im demografischen Wandel in Deutschland - Potenziale zur Einbindung älterer Menschen in zukünftige Lebenswelten / Liesa Anneke König
https://nbn-resolving.org/urn:nbn:de:hbz:468-20191023-092155-6
322: Das Spiel mit der Grenze - Führung im Spannungsfeld von Formalität und Informalität / Timo Hillenbrand ; Gutachter: Andreas Göbel, Veronika Tacke
https://nbn-resolving.org/urn:nbn:de:bvb:20-opus-189290
323: Lösen von Alltagsproblemen, Kreativität und Sozialkompetenz : interkulturell und intergenerational vergleichende Studie zwischen Deutschland und Südkorea / Seon Joo Born ; Betreuer: Willie van Peer
https://nbn-resolving.org/urn:nbn:de:bvb:19-251408
324: Kunstschutz im besetzten Deutschland : Restitution und Kulturpolitik in der französischen und amerikanischen Besatzungszone (1944–1953) / Emily Löffler
https://nbn-r

382: Verlebendigung der Technik - Perspektiven im feministischen Technikdiskurs / Angelika Saupe ; Gutachter: Marlies Krüger, Maria Osietzki ; Betreuer: Marlies Krüger
https://nbn-resolving.org/urn:nbn:de:gbv:46-00107603-19
383: Wissenschaft und Gefühl : Eine Interviewstudie zum Umgang mit Empfindungen in der Forschung an nichtmenschlichen Primaten / Julia Keil
https://nbn-resolving.org/urn:nbn:de:kobv:188-refubium-25348-7
384: Gottschee revisited : Geschichtsnarrative und Identitätsmanagement im Cyberspace / Georg Marschnig
https://nbn-resolving.org/urn:nbn:de:101:1-2019082509060373798770
385: Die Abwertung des Anderen mittels Sprache im Amateurfußball : Ein soziolinguistischer Vergleich in Deutschland und Frankreich / Florian Koch
https://nbn-resolving.org/urn:nbn:de:101:1-2019082509002486598350
386: Russian cities in motion: the Marshrutka as a multifaceted issue of Post-Soviet urban life / Tonio Weicker ; Gutachter: Hans-Liudger Dienel, Oksana Zaporozhets ; Hans-Liudger Dienel, Oks

...

568: "Aus dem Volkskörper entfernt"? : Homosexuelle Männer im Nationalsozialismus / Alexander Zinn
https://nbn-resolving.org/urn:nbn:de:101:1-2018070311460598838759
569: Der Richter und sein Tagebuch : Eugen Wilhelm als Elsässer und homosexueller Aktivist im deutschen Kaiserreich / Kevin Dubout
https://nbn-resolving.org/urn:nbn:de:101:1-2018070311454582921471
570: Transnationaler Aktivismus und Frauenarbeit : Social Movement Unionism in Bangladesch / Elisabeth Fink
https://nbn-resolving.org/urn:nbn:de:101:1-2018070311442348836452
571: Internationalität als Herausforderung und Chance : Eine organisationstheoretische Betrachtung des Diversity Managements an deutschen Hochschulen am Beispiel von internationalen Masterstudiengängen im Bereich der Agrarwissenschaften / Constanze Engel
https://nbn-resolving.org/urn:nbn:de:0002-405117
572: Das Sicherheitsdispositiv der Resilienz : Katastrophische Risiken und die Biopolitik vitaler Systeme / Andreas Folkers
https://nbn-resolving.org/urn:nbn:de

632: Zwischen Antisemitismus und Apartheid : der Kampf jüdischer Südafrikaner gegen die ’Rassen’herrschaft (1948-1990) / Hanno Plass ; Gutachter: Stefanie Schüler-Springorum, Axel Schildt ; Stefanie Schüler-Springorum, Axel Schildt
https://nbn-resolving.org/urn:nbn:de:101:1-201804135731
633: Ressourcenorientierte Zugehörigkeitsverortungen - Eine qualitative Untersuchung zu Selbstverortungen von Frauen mit italienisch tradierter Zuwanderungsgeschichte im urbanen Kontext / Ronja Müller-Kalkstein
https://nbn-resolving.org/urn:nbn:de:hebis:34-2017120853938
634: Analyse diffusiver und intermittierender Suchstrategien mithilfe neuer Monte-Carlo-Algorithmen für allgemeine Reaktions-Diffusionsprobleme / Karsten Schwarz ; Betreuer: Heiko Rieger
https://nbn-resolving.org/urn:nbn:de:bsz:291-scidok-ds-269700
635: Die Einheit der Vielfalt : Zur Institutionalisierung der globalen Kategorie "indigene Völker" / Hannah Bennani
https://nbn-resolving.org/urn:nbn:de:101:1-201802132948
636: Auswirkungen vo

699: Lehramtsstudierende analysieren Praxis. Ein Vergleich der Effekte unterschiedlicher fallbasierter Lehr-Lern-Arrangements / Jürgen Schneider ; Betreuer: Thorsten Bohl
https://nbn-resolving.org/urn:nbn:de:bsz:21-dspace-718431
700: Deutsch in Santa Cruz (Do Sul) : Studium und Analyse der gegenwärtigen Situation der Sprache / Lissi Bender ; Betreuer: Reinhard Johler
https://nbn-resolving.org/urn:nbn:de:bsz:21-dspace-718327
701: Subjektive Gesundheit im Nachwuchsleistungssport / Alexia Schnell ; Betreuer: Ansgar Thiel
https://nbn-resolving.org/urn:nbn:de:bsz:21-dspace-715368
702: Wo zuhause? Wo fremd? Die Russlanddeutschen zwischen National- und Kulturgrenzen / Gerald Gredinger ; Betreuer: Michael Schönhuth
https://nbn-resolving.org/urn:nbn:de:hbz:385-9810
703: Gesundheit im Habitus von Führungskräften. Eine rekonstruktive Studie anhand von Interviews mit Führungskräften / Verena Hellfritsch ; Betreuer: Ansgar Thiel
https://nbn-resolving.org/urn:nbn:de:bsz:21-dspace-677723
704: Code-Sw

777: Fachliche Publikationskulturen und Open Access. Fächerübergreifende Entwicklungstendenzen und Spezifika der Erziehungswissenschaft und Bildungsforschung. / Doris Bambey. Betreuer: Rudi Schmiede ; Andrea Rapp
https://nbn-resolving.org/urn:nbn:de:tuda-tuprints-56032
778: Die soziale Organisiertheit und Organisierbarkeit von Interessen(freiheit) – Der Fall der managerialen Governance akademischer Lehrtätigkeit / Christian Johann Schmid. Betreuer: Uwe Wilkesmann. Gutachter: Jens Rowold
https://nbn-resolving.org/urn:nbn:de:101:1-201608261976
779: Zentralörtliche Funktionsräume : Empirische Ermittlung und konzeptionelle Ausgestaltung am Beispiel der Planungsregion Düsseldorf / Florian Flex. Betreuer: Stefan Greiving. Gutachter: Rainer Winkel
https://nbn-resolving.org/urn:nbn:de:101:1-201608261803
780: B2B-Kommunikation: Mit der Social Media Spirale zum sozialen Unternehmen : eine Betrachtung unter besonderer Berücksichtigung einer konstruktivistischen Perspektive / Markus Knetsch
https:

824: Klimaschutz als Medienwirkung : eine kommunikationswissenschaftliche Studie zur Konzeption, Rezeption und Wirkung eines Online-Spiels zum Stromsparen / Imke Hoppe. Betreuer: Jens Wolling. Gutachter: Elke Grittmann ; Jeffrey Wimmer
https://nbn-resolving.org/urn:nbn:de:gbv:ilm1-2014000648
825: Schlagen und Wogen / Uwe Wippich. Gutachter: Anna Tuschling ; Astrid Deuber-Mankowsky
https://nbn-resolving.org/urn:nbn:de:hbz:294-46880
826: Kooperation, Beziehungsstrukturen und Sozialkapital von Lehrkräften : eine Analyse zum Programm „Jedem Kind ein Instrument“ / Sabrina Kulin. Betreuer: Knut Schwippert
https://nbn-resolving.org/urn:nbn:de:gbv:18-78159
827: Klimakatastrophen, Vertreibung und Gewalt : eine makro-qualitative Untersuchung sowie eine Einzelfallstudie über den Zusammenhang von umweltbedingten Bevölkerungsbewegungen und gewaltsamen Konflikten / Andreas Bernath. Betreuer: Michael Brzoska
https://nbn-resolving.org/urn:nbn:de:gbv:18-78239
828: Kulturkontakt am Altar. Silberarbeiten